Найти решение краевой задачи для одномерного стационарного уравнения теплопроводности
$$\frac{d}{dx}[k(x)\frac{du}{dx}] - q(x)u = -f(x)$$
в 11 равноудаленных точках отрезка $[0, 1]$ с относительной точностью $0.0001$.
Отладку программы произвести на модельной задаче с постоянными коэффициентами.

In [42]:
import numpy as np
from pandas import DataFrame

Условие. Задача 4.

Краевые условия:
$$u(0) = 1; u(1) = 0$$
\begin{equation*}
 \begin{cases}
    u(x_0 - 0) = u(x_0 + 0) \\
    k(x_0 - 0) u_{x}(x_0 - 0) = k(x_0 + 0) u_{x}(x_0 + 0) 
 \end{cases}
\end{equation*}
$$x < x_0 = 0.525 : k(x) = x^2 + 1; q(x) = e^{-x}; f(x) = 1$$
$$x > x_0: k(x) = x; q(x) = e^{-x}; f(x) = x^3$$

Модельная задача:
$$x_0 = 1 / \sqrt{2} : k(x) = k(x_0); q(x) = q(x_0); f(x) = f(x_0)$$

In [43]:
def k(t : float, t0: float, flag = 0) -> float:
    if t < t0:
        return t**2
    elif t > t0:
        return t
    else:
        if flag == 0:
            return t**2
        else: 
            return t


def q(t : float, t0 : float, flag = 0) -> float:
    return np.exp(-t)

def f(t : float, t0 : float, flag = 0) -> float:
    if t < t0:
        return 1
    elif t > t0:
        return t**3
    else:
        if flag == 0:
            return 1
        else: 
            return t**3

In [44]:
#условия
accur = 10**(-4)

u0 = 0
u1 = 1

x_0 = 0.525

k1 = k(x_0, x_0)
k2 = k(x_0, x_0, 1)

q1 = q(x_0, x_0)
q2 = q(x_0, x_0, 1)

f1 = f(x_0, x_0)
f2 = f(x_0, x_0, 1)

#разделения отрезка на n частей
L = 11

x = np.round(np.linspace(0.0, 1.0, L), 7)



Аналитическое решение модельной задачи

In [45]:
#решение характерестического уравнения
lambda1 = (q1 / k1)**0.5
lambda2 = (q2 / k2)**0.5

#частные решения
mu1 = f1 / q1
mu2 = f2 / q2

#вспомогательные коэффициенты
a11 = np.exp(-lambda1 * x_0) - np.exp(lambda1 * x_0)
a12 = np.exp(lambda2 * (2 - x_0)) - np.exp(lambda2 * x_0)
a21 = k1 * lambda1 * (np.exp(-lambda1 * x_0) + np.exp(lambda1 * x_0))
a22 = k2 * lambda2 * (np.exp(lambda2 * (2 - x_0)) + np.exp(lambda2 * x_0)) 

b1 = mu2 - mu1 + (mu1 - u0) * np.exp(lambda1 * x_0) - (mu2 - u1) * np.exp(lambda2 * (1 - x_0))
b2 = k1 * lambda1 * (u0 - mu1) * np.exp(lambda1 * x_0) + k2 * lambda2 * (u1 - mu2) * np.exp(lambda2 * (1 - x_0))

#коэффициенты для общего решения
c1 = (((u0 - mu1) * a11 - b1) * a22 - ((u0 - mu1) * a21 - b2) * a12) / (a11 * a22 - a12 * a21) 
c2 = (b1 * a22 - b2 * a12) / (a11 * a22 - a12 * a21)
c3 = (b2 * a11 - b1 * a21) / (a11 * a22 - a12 * a21)
c4 = (u1 - mu2) * np.exp(lambda2) - c3 * np.exp(2 * lambda2)

#нахождение аналитического решения
u_alpha = np.round(c1 * np.exp(lambda1 * x[x < x_0]) + c2 * np.exp(-lambda1 * x[x < x_0]) + mu1, 7)
u_beta = np.round(c3 * np.exp(lambda2 * x[x > x_0]) + c4 * np.exp(-lambda2 * x[x > x_0]) + mu2, 7)


u_analytical = list(u_alpha) + list(u_beta)


Численное решение модельной задачи с заданной степенью точности(Метод встречных прогонок)

In [46]:
h = 1 / L
l_1 = int(x_0 / h)
l_2 = int(np.round(x_0 / h))

if l_1 == l_2:
    l_2 += 1

# e = l = [1, l_1 -1]
ae = k1
be = -2 * k1 - q1 * h**2 
ce = k1
de = -f1 * h**2

#r = l = [L - 1; l_2 + 1]
ar = k2
br = -2 * k2 - q2 * h**2
cr = k2
dr = -f2 * h**2

#прямая прогонка
alpha1 = -ae / be
beta1 = (de - ce * u0) / be

alphaL = -cr / br 
betaL = (dr - cr * u1) / br

coefs1 = [[alpha1, beta1]]
coefs2 = [[alphaL, betaL]]

for i in range(1, l_1 - 1):
    coefs1.append([-ae / (be + ce * coefs1[i - 1][0]), (de - ce * coefs1[i-1][1]) / (be + ce * coefs1[i-1][0])])

for i in range(1, L - l_2 - 1):
    coefs2.append([-cr / (br + ar * coefs2[i - 1][0]), (dr - ar * coefs2[i - 1][1]) / (br + ar * coefs2[i - 1][0])])

u_l_ab = (k1 * coefs1[-1][1] + k2 * coefs2[0][1]) / (k1 * (1 - coefs1[-1][0]) + k2 * (1 - coefs2[0][0]))


#обратная прогонка

coefs1.reverse()
u_a = [u_l_ab, coefs1[0][0] * u_l_ab + coefs1[0][1]]
for i in range(2, l_1):
    u_a.append(coefs1[i - 1][0] * u_a[i - 1] + coefs1[i - 1][1])
u_a.append(u0)
u_a.reverse()


u_b = [u_l_ab, coefs2[0][0] * u_l_ab + coefs2[0][1]]
for i in range(2, L - l_2 - 1):
    u_b.append(coefs2[i - 1][0] * u_b[i - 1] + coefs2[i - 1][1])
u_b.append(u1)
u_sweep = u_a + u_b

Численное решение задачи с переменными коэффициентами 

In [47]:
def a_l(t : float) -> float:
    return k(t + h/2, x_0)

def b_l(t : float) -> float:
    return -(k(t + h/2, x_0) + k(t - h/2, x_0) + q(t, x_0) * h**2)

def c_l(t : float) -> float:
    return k(t - h/2, x_0)

def d_l(t : float) -> float:
    return -f(t, x_0) * h**2 

alpha1 = -a_l(x[1]) / b_l(x[1])
beta1 = (d_l(x[1]) - c_l(x[1]) * u0) / b_l(x[1])

alphaL1 = -a_l(x[-2]) / b_l(x[-2]) 
betaL1 = (d_l(x[-2]) - c_l(x[-2]) * u1) / b_l(x[-2])

coefs11 = [[alpha1, beta1]]
coefs22 = [[alphaL1, betaL1]]

for i in range(1, l_1 - 1):
    coefs11.append([-a_l(x[i + 1]) / (b_l(x[i + 1]) + c_l(x[i + 1]) * coefs11[i - 1][0]), (d_l(x[i + 1]) - c_l(x[i + 1]) * coefs11[i-1][1]) / (b_l(x[i + 1]) + c_l(x[i + 1]) * coefs11[i - 1][0])])

for i in range(1, L - l_2 - 1):
    coefs22.append([-c_l(x[-i - 2]) / (b_l(x[-i - 2]) + a_l(x[-i - 2]) * coefs22[i - 1][0]), (d_l(x[-i - 2]) - a_l(x[-i - 2]) * coefs22[i - 1][1]) / (b_l(x[-i - 2]) + a_l(x[-i - 2]) * coefs22[i - 1][0])])

u_l_ab = (k1 * coefs11[-1][1] + k2 * coefs22[0][1]) / (k1 * (1 - coefs11[-1][0]) + k2 * (1 - coefs22[0][0]))


#обратная прогонка

coefs11.reverse()
u_a1 = [u_l_ab]
for i in range(1, l_1):
    u_a1.append(coefs11[i - 1][0] * u_a1[i - 1] + coefs11[i - 1][1])
u_a1.append(u0)
u_a1.reverse()


u_b1 = [u_l_ab]
for i in range(1, L - l_2 - 1):
    u_b1.append(coefs22[i - 1][0] * u_b1[i - 1] + coefs22[i - 1][1])
u_b1.append(u1)

u_sweep_tmp = u_a1 + u_b1

In [48]:
dat = {
    'x' : x,
    'Analytical solve' : u_analytical,
    'Sweep solve' : u_sweep,
    'Error' : np.array(u_analytical) - np.array(u_sweep),
    'Max error' : np.max(np.array(u_analytical) - np.array(u_sweep)),
    'Sweep tmp solve' : u_sweep_tmp
}
df = DataFrame(data=dat)
df

,x,Analytical solve,Sweep solve,Error,Max error,Sweep tmp solve
0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
1,0.1,0.200937,0.217055,-0.016117,0.0,0.956445
2,0.2,0.369849,0.407975,-0.038126,0.0,1.038377
3,0.3,0.510367,0.576147,-0.065780,0.0,1.050303
4,0.4,0.625511,0.724554,-0.099043,0.0,1.041409
5,0.5,0.717759,0.855828,-0.138069,0.0,1.023201
6,0.6,0.769137,0.855828,-0.086692,0.0,1.023201
7,0.7,0.816549,0.924747,-0.108198,0.0,1.013641
8,0.8,0.870413,0.943248,-0.072836,0.0,1.010392
9,0.9,0.931334,0.947502,-0.016168,0.0,1.007253
